# Contribution to Active Risk (CAR): Credit vs. Credit+Idio Worlds

We define:

- Node-level credit active exposure:  
  $ f_{g,c} \in \mathbb{R}^{K_c} $

- Portfolio credit active exposure:  
  $ f_c = \sum_g f_{g,c} $

- Credit factor covariance:  
  $ \Sigma_c \in \mathbb{R}^{K_c \times K_c} $

- Node-level idiosyncratic SAR (exogenous):  
  $ \sigma_{\epsilon,g} $

---

## Portfolio-level risk measures

- Credit-only risk:  
  $$
  \sigma_{credit} = \sqrt{f_c^\top \Sigma_c f_c}
  $$

- Idiosyncratic-only risk:  
  $$
  \sigma_{idio} = \sqrt{\sum_g \sigma_{\epsilon,g}^2}
  $$

- Credit+Idio-only risk:  
  $$
  \sigma_{credit+idio} = \sqrt{ \sigma_{credit}^2 + \sigma_{idio}^2 }
  $$

---

## Marginal vectors (gradients)

- Credit-only marginal:  
  $$
  m_{credit} = \frac{ \Sigma_c f_c }{ \sigma_{credit} }
  $$

- Credit+Idio-only marginal:  
  $$
  m_{credit+idio} = \frac{ \Sigma_c f_c }{ \sigma_{credit+idio} }
  $$

---

## Node-level CARs

For each node $g$:

- Credit-only CAR:  
  $$
  \text{CAR}^{credit}_g = f_{g,c}^\top m_{credit}
  $$

- Credit+Idio-only CAR:  
  $$
  \text{CAR}^{credit+idio}_g = f_{g,c}^\top m_{credit+idio} + \frac{\sigma_{\epsilon,g}^2}{\sigma_{credit+idio}}
  $$

- Idio CAR (explicit):  
  $$
  \text{CAR}^{idio}_g = \frac{\sigma_{\epsilon,g}^2}{\sigma_{credit+idio}}
  $$

---

## Beta / Residual Split

Let $b_c$ be the benchmark credit exposure vector.  
Decompose $f_{g,c}$ into:

- Beta-aligned part:  
  $$
  f_{g,\beta} = \alpha_g b_c, 
  \quad \alpha_g = \frac{ b_c^\top \Sigma_c f_{g,c} }{ b_c^\top \Sigma_c b_c }
  $$

- Residual part:  
  $$
  f_{g,res} = f_{g,c} - f_{g,\beta}
  $$

with $ f_{g,res}^\top \Sigma_c b_c = 0 $.

Then:

- $ \text{CAR}^{credit}_{g} = \text{CAR}^{\beta}_g + \text{CAR}^{res}_g $
- $ \text{SAR}^{credit}_{g}{}^2 = \text{SAR}^{\beta}_{g}{}^2 + \text{SAR}^{res}_{g}{}^2 $

---

## Additivity Checks

- $$
  \sum_g \text{CAR}^{credit}_g = \sigma_{credit}
  $$

- $$
  \sum_g \text{CAR}^{credit+idio}_g = \sigma_{credit+idio}
  $$

- $$
  \sigma_{credit+idio}^2 = \sigma_{\beta}^2 + \sigma_{res}^2 + \sigma_{idio}^2
  $$



In [ ]:

"""
credit_idio_only_demo.py
------------------------
Single-file module that:
  - Builds a node-level CAR/SAR table with MultiIndex column groups:
      * credit_idio_only (credit factors + idiosyncratic; excludes rates)
      * credit_only      (credit factors only)
      * SAR              (standalone diagnostics)
  - Splits credit CAR into beta and residual via Mahalanobis projection
    onto the benchmark credit exposure direction.
  - Includes runnable examples under __main__ with toy data and checks.

Author: ChatGPT
License: MIT
"""
from __future__ import annotations

import numpy as np
import pandas as pd
from dataclasses import dataclass
from typing import Dict, Optional

# ======================================================================
# Data containers
# ======================================================================

@dataclass
class PortfolioBlocks:
    sigma_credit_only: float
    mcar_credit_only_vector: np.ndarray
    sigma_idiosyncratic_only: float
    sigma_credit_idio_only: float
    mcar_credit_idio_only_vector: np.ndarray

@dataclass
class ProjectionResult:
    alpha_scalar: float
    node_beta_exposure_vector: np.ndarray
    node_residual_exposure_vector: np.ndarray
    beta_direction_quadratic_form: float
    orthogonality_check_value: float

# ======================================================================
# Utilities
# ======================================================================

def _as_numpy_array(x) -> np.ndarray:
    if isinstance(x, pd.Series):
        return x.to_numpy(dtype=float)
    if isinstance(x, pd.DataFrame):
        return x.to_numpy(dtype=float)
    return np.asarray(x, dtype=float)

def _symmetrize_matrix(matrix: np.ndarray) -> np.ndarray:
    return 0.5 * (matrix + matrix.T)

def _safe_scalar(value) -> float:
    return float(np.asarray(value, dtype=float).reshape(-1)[0])

# ======================================================================
# Prep: restrict to credit factors & form active exposures
# =========

def prepare_credit_space(
    portfolio_exposure_by_node_df: pd.DataFrame,
    benchmark_exposure_by_node_df: pd.DataFrame,
    factor_covariance_df: pd.DataFrame,
    credit_factor_mask: pd.Series,
) -> Dict:
    """
    Align shapes, restrict to credit factors, and compute node and portfolio credit-active exposures.

    Returns dict with:
      - Sigma_c (np.ndarray, Kc×Kc): credit-only covariance
      - node_credit_active_exposure_matrix (np.ndarray, G×Kc): f_{g,c}
      - portfolio_credit_active_exposure_vector (np.ndarray, Kc,): f_c
      - node_index (pd.Index)
      - credit_factor_index (list[str])
    """
    # Align columns across inputs (use covariance ordering as canonical)
    common_factors = factor_covariance_df.index.tolist()
    portfolio_exposure_by_node_df = portfolio_exposure_by_node_df.loc[:, common_factors]
    benchmark_exposure_by_node_df = benchmark_exposure_by_node_df.loc[:, common_factors]

    # Credit factor names in correct order
    credit_factor_names = [
        f for f in common_factors
        if bool(credit_factor_mask.loc[f])
    ]

    # Restrict to credit block
    Sigma_c_df = factor_covariance_df.loc[credit_factor_names, credit_factor_names]
    Sigma_c = _symmetrize_matrix(Sigma_c_df.to_numpy(dtype=float))

    beta_p_c_df = portfolio_exposure_by_node_df.loc[:, credit_factor_names]
    beta_b_c_df = benchmark_exposure_by_node_df.loc[:, credit_factor_names]

    # Node credit-active exposures
    node_credit_active_exposure_matrix = (beta_p_c_df - beta_b_c_df).to_numpy(dtype=float)
    portfolio_credit_active_exposure_vector = node_credit_active_exposure_matrix.sum(axis=0)

    return {
        "Sigma_c": Sigma_c,
        "node_credit_active_exposure_matrix": node_credit_active_exposure_matrix,
        "portfolio_credit_active_exposure_vector": portfolio_credit_active_exposure_vector,
        "node_index": portfolio_exposure_by_node_df.index.copy(),
        "credit_factor_index": credit_factor_names,
    }

# ======================================================================
# Portfolio-level blocks (credit_only and credit_idio_only worlds)
# ======================================================================

def compute_portfolio_blocks_for_credit_and_idio_only(
    portfolio_credit_active_exposure_vector: np.ndarray,
    Sigma_c: np.ndarray,
    idiosyncratic_sar_by_node: pd.Series,
) -> PortfolioBlocks:
    """
    Compute portfolio-level SARs and marginal vectors for the two worlds:
      - credit_only:        denominator σ_c
      - credit_idio_only:   denominator √(σ_c^2 + σ_idio^2)
    """
    f_c = _as_numpy_array(portfolio_credit_active_exposure_vector).reshape(-1, 1)
    credit_variance = _safe_scalar(f_c.T @ Sigma_c @ f_c)
    sigma_credit_only = float(np.sqrt(max(credit_variance, 0.0)))

    if sigma_credit_only > 0:
        mcar_credit_only_vector = (Sigma_c @ f_c) / sigma_credit_only
    else:
        mcar_credit_only_vector = np.zeros_like(f_c)

    idio_variance_total = float(np.sum(np.square(idiosyncratic_sar_by_node.astype(float).to_numpy())))
    sigma_idiosyncratic_only = float(np.sqrt(max(idio_variance_total, 0.0)))

    sigma_credit_idio_only = float(np.sqrt(credit_variance + idio_variance_total))

    if sigma_credit_idio_only > 0:
        mcar_credit_idio_only_vector = (Sigma_c @ f_c) / sigma_credit_idio_only
    else:
        mcar_credit_idio_only_vector = np.zeros_like(f_c)

    return PortfolioBlocks(
        sigma_credit_only=sigma_credit_only,
        mcar_credit_only_vector=mcar_credit_only_vector.reshape(-1),
        sigma_idiosyncratic_only=sigma_idiosyncratic_only,
        sigma_credit_idio_only=sigma_credit_idio_only,
        mcar_credit_idio_only_vector=mcar_credit_idio_only_vector.reshape(-1),
    )

# ======================================================================
# Beta projection (Mahalanobis under Σ_c)
# ======================================================================

def choose_benchmark_credit_beta_direction(
    benchmark_exposure_by_node_df: pd.DataFrame,
    credit_factor_names: list[str],
) -> np.ndarray:
    """
    Use the top-level benchmark credit exposure vector as the beta direction b_c (Kc,).
    """
    benchmark_credit_exposure_vector = benchmark_exposure_by_node_df.loc[:, credit_factor_names].sum(axis=0).to_numpy(dtype=float)
    return benchmark_credit_exposure_vector

def project_node_exposure_onto_beta_under_credit_metric(
    node_credit_active_exposure_vector: np.ndarray,
    beta_direction_vector: np.ndarray,
    Sigma_c: np.ndarray,
) -> ProjectionResult:
    """
    Mahalanobis projection of a node's credit exposure onto the beta direction under Σ_c.
    Returns projected (beta) and residual components, plus diagnostics.
    """
    f_g = _as_numpy_array(node_credit_active_exposure_vector).reshape(-1, 1)
    b_c = _as_numpy_array(beta_direction_vector).reshape(-1, 1)

    Sb = Sigma_c @ b_c
    beta_direction_quadratic_form = _safe_scalar(b_c.T @ Sb)
    if np.isclose(beta_direction_quadratic_form, 0.0):
        raise ValueError("Beta direction has zero variance under Σ_c (b_c' Σ_c b_c == 0).")

    alpha_scalar = _safe_scalar((b_c.T @ Sigma_c @ f_g) / beta_direction_quadratic_form)
    node_beta_exposure_vector = alpha_scalar * b_c
    node_residual_exposure_vector = f_g - node_beta_exposure_vector
    orthogonality_check_value = _safe_scalar(node_residual_exposure_vector.T @ Sigma_c @ b_c)

    return ProjectionResult(
        alpha_scalar=float(alpha_scalar),
        node_beta_exposure_vector=node_beta_exposure_vector.reshape(-1),
        node_residual_exposure_vector=node_residual_exposure_vector.reshape(-1),
        beta_direction_quadratic_form=float(beta_direction_quadratic_form),
        orthogonality_check_value=float(orthogonality_check_value),
    )

# ======================================================================
# Per-node metrics
# ======================================================================

def compute_node_metrics_row(
    node_credit_active_exposure_vector: np.ndarray,
    beta_direction_vector: np.ndarray,
    Sigma_c: np.ndarray,
    mcar_credit_only_vector: np.ndarray,
    mcar_credit_idio_only_vector: np.ndarray,
    node_idiosyncratic_sar: float,
) -> Dict[tuple, float]:
    """
    Compute the requested outputs for a single node.
    Returns a dict keyed by (parent, child) to form a MultiIndex later.
    """
    proj = project_node_exposure_onto_beta_under_credit_metric(
        node_credit_active_exposure_vector=node_credit_active_exposure_vector,
        beta_direction_vector=beta_direction_vector,
        Sigma_c=Sigma_c,
    )

    f_g = _as_numpy_array(node_credit_active_exposure_vector)
    f_beta = proj.node_beta_exposure_vector
    f_res = proj.node_residual_exposure_vector

    # Credit-only CARs (denominator = sigma_credit_only)
    car_credit_only = float(f_g @ mcar_credit_only_vector)
    car_credit_only_beta = float(f_beta @ mcar_credit_only_vector)
    car_credit_only_res = float(f_res @ mcar_credit_only_vector)

    # Credit+Idio-only CARs (denominator = sigma_credit_idio_only)
    car_credit = float(f_g @ mcar_credit_idio_only_vector)
    car_credit_beta = float(f_beta @ mcar_credit_idio_only_vector)
    car_credit_res = float(f_res @ mcar_credit_idio_only_vector)

    # SAR diagnostics (standalone)
    Sigma_c_np = _as_numpy_array(Sigma_c)
    sar_credit = float(np.sqrt(max(_safe_scalar(f_g.reshape(1,-1) @ Sigma_c_np @ f_g.reshape(-1,1)), 0.0)))
    sar_beta = float(np.sqrt(max(_safe_scalar(f_beta.reshape(1,-1) @ Sigma_c_np @ f_beta.reshape(-1,1)), 0.0)))
    sar_res = float(np.sqrt(max(_safe_scalar(f_res.reshape(1,-1) @ Sigma_c_np @ f_res.reshape(-1,1)), 0.0)))
    sar_idio = float(node_idiosyncratic_sar)

    return {
        # credit_idio_only (we will add car_idio and car_total at table assembly time)
        ("credit_idio_only", "car_credit"): car_credit,
        ("credit_idio_only", "car_beta"): car_credit_beta,
        ("credit_idio_only", "car_res"): car_credit_res,

        # credit_only
        ("credit_only", "car_total"): car_credit_only,
        ("credit_only", "car_beta"): car_credit_only_beta,
        ("credit_only", "car_res"): car_credit_only_res,

        # SARs
        ("SAR", "sar_credit"): sar_credit,
        ("SAR", "sar_beta"): sar_beta,
        ("SAR", "sar_res"): sar_res,
        ("SAR", "sar_idio"): sar_idio,
    }

# ======================================================================
# Build full table
# ======================================================================

def build_credit_idio_only_table(
    portfolio_exposure_by_node_df: pd.DataFrame,
    benchmark_exposure_by_node_df: pd.DataFrame,
    factor_covariance_df: pd.DataFrame,
    credit_factor_mask: pd.Series,
    idiosyncratic_sar_by_node: pd.Series,
    beta_direction_vector_optional: Optional[np.ndarray] = None,
    add_totals_row: bool = True,
    audit_checks: bool = True,
    tolerance: float = 1e-8,
) -> pd.DataFrame:
    """
    Return MultiIndex DataFrame with grouped columns for:
      - credit_idio_only: car_total, car_credit, car_beta, car_res, car_idio
      - credit_only:      car_total, car_beta, car_res
      - SAR:              sar_credit, sar_beta, sar_res, sar_idio
    """
    prep = prepare_credit_space(
        portfolio_exposure_by_node_df=portfolio_exposure_by_node_df,
        benchmark_exposure_by_node_df=benchmark_exposure_by_node_df,
        factor_covariance_df=factor_covariance_df,
        credit_factor_mask=credit_factor_mask,
    )
    Sigma_c = prep["Sigma_c"]
    node_matrix = prep["node_credit_active_exposure_matrix"]
    f_c = prep["portfolio_credit_active_exposure_vector"]
    node_index = prep["node_index"]
    credit_factor_names = prep["credit_factor_index"]

    # Portfolio-level blocks
    blocks = compute_portfolio_blocks_for_credit_and_idio_only(
        portfolio_credit_active_exposure_vector=f_c,
        Sigma_c=Sigma_c,
        idiosyncratic_sar_by_node=idiosyncratic_sar_by_node,
    )

    # Beta direction
    if beta_direction_vector_optional is None:
        beta_direction_vector = choose_benchmark_credit_beta_direction(
            benchmark_exposure_by_node_df=benchmark_exposure_by_node_df,
            credit_factor_names=credit_factor_names,
        )
    else:
        beta_direction_vector = beta_direction_vector_optional

    # Node loop
    rows = []
    for i, node_name in enumerate(node_index):
        node_idio_sar = float(idiosyncratic_sar_by_node.loc[node_name])
        row = compute_node_metrics_row(
            node_credit_active_exposure_vector=node_matrix[i, :],
            beta_direction_vector=beta_direction_vector,
            Sigma_c=Sigma_c,
            mcar_credit_only_vector=blocks.mcar_credit_only_vector,
            mcar_credit_idio_only_vector=blocks.mcar_credit_idio_only_vector,
            node_idiosyncratic_sar=node_idio_sar,
        )

        # Idio CAR and total CAR in credit_idio_only world (denominator = sigma_credit_idio_only)
        car_idio = (node_idio_sar ** 2) / blocks.sigma_credit_idio_only if blocks.sigma_credit_idio_only > 0 else 0.0
        car_total = row[("credit_idio_only", "car_credit")] + car_idio

        row[("credit_idio_only", "car_idio")] = float(car_idio)
        row[("credit_idio_only", "car_total")] = float(car_total)

        rows.append((node_name, row))

    # Assemble DataFrame with MultiIndex columns
    df_out = pd.DataFrame.from_dict({name: vals for name, vals in rows}, orient="index")

    # Ensure canonical column order
    desired_columns = [
        ("credit_idio_only", "car_total"),
        ("credit_idio_only", "car_credit"),
        ("credit_idio_only", "car_beta"),
        ("credit_idio_only", "car_res"),
        ("credit_idio_only", "car_idio"),
        ("credit_only", "car_total"),
        ("credit_only", "car_beta"),
        ("credit_only", "car_res"),
        ("SAR", "sar_credit"),
        ("SAR", "sar_beta"),
        ("SAR", "sar_res"),
        ("SAR", "sar_idio"),
    ]
    available = [c for c in desired_columns if c in df_out.columns]
    df_out = df_out[available]
    df_out.index.name = "node"

    # Portfolio summary
    portfolio_summary = {}
    portfolio_summary["sigma_credit_only"] = blocks.sigma_credit_only
    portfolio_summary["sigma_idiosyncratic_only"] = blocks.sigma_idiosyncratic_only
    portfolio_summary["sigma_credit_idio_only"] = blocks.sigma_credit_idio_only

    # Euler sums
    portfolio_summary["sum_car_credit_only"] = float(df_out[("credit_only", "car_total")].sum())
    portfolio_summary["sum_car_credit_idio_only"] = float(df_out[("credit_idio_only", "car_total")].sum())

    # Rebuild SAR totals from node SAR columns (variance additivity)
    sigma_credit_from_nodes = float(np.sqrt(max(np.sum(np.square(df_out[("SAR", "sar_credit")].to_numpy())), 0.0)))
    sigma_idio_from_nodes = float(np.sqrt(max(np.sum(np.square(df_out[("SAR", "sar_idio")].to_numpy())), 0.0)))
    sigma_credit_idio_via_nodes = float(np.sqrt(
        max(np.sum(np.square(df_out[("SAR", "sar_credit")].to_numpy())), 0.0) +
        max(np.sum(np.square(df_out[("SAR", "sar_idio")].to_numpy())), 0.0)
    ))

    portfolio_summary["sigma_credit_only_from_nodes"] = sigma_credit_from_nodes
    portfolio_summary["sigma_idio_only_from_nodes"] = sigma_idio_from_nodes
    portfolio_summary["sigma_credit_idio_only_from_nodes"] = sigma_credit_idio_via_nodes

    # Attach attrs
    df_out.attrs["portfolio_summary"] = portfolio_summary

    # Optional totals row
    if add_totals_row:
        totals = pd.Series(index=df_out.columns, dtype=float)
        totals[("credit_idio_only", "car_total")] = blocks.sigma_credit_idio_only
        totals[("credit_idio_only", "car_credit")] = df_out[("credit_idio_only", "car_credit")].sum()
        totals[("credit_idio_only", "car_beta")] = df_out[("credit_idio_only", "car_beta")].sum()
        totals[("credit_idio_only", "car_res")] = df_out[("credit_idio_only", "car_res")].sum()
        totals[("credit_idio_only", "car_idio")] = df_out[("credit_idio_only", "car_idio")].sum()

        totals[("credit_only", "car_total")] = blocks.sigma_credit_only
        totals[("credit_only", "car_beta")] = df_out[("credit_only", "car_beta")].sum()
        totals[("credit_only", "car_res")] = df_out[("credit_only", "car_res")].sum()

        totals[("SAR", "sar_credit")] = np.sqrt(np.sum(np.square(df_out[("SAR", "sar_credit")].to_numpy())))
        totals[("SAR", "sar_beta")] = np.sqrt(np.sum(np.square(df_out[("SAR", "sar_beta")].to_numpy())))
        totals[("SAR", "sar_res")] = np.sqrt(np.sum(np.square(df_out[("SAR", "sar_res")].to_numpy())))
        totals[("SAR", "sar_idio")] = np.sqrt(np.sum(np.square(df_out[("SAR", "sar_idio")].to_numpy())))

        df_out.loc["TOTAL"] = totals

    # Basic audits
    if audit_checks and add_totals_row:
        # Sum of credit-only CARs equals portfolio credit SAR
        assert np.isclose(
            df_out.loc["TOTAL", ("credit_only", "car_total")],
            portfolio_summary["sigma_credit_only"],
            rtol=1e-6, atol=1e-9
        ), "Sum of credit_only car_total does not equal sigma_credit_only."
        # Sum of credit+idio-only CARs equals portfolio credit+idio SAR
        assert np.isclose(
            df_out.loc["TOTAL", ("credit_idio_only", "car_total")],
            portfolio_summary["sigma_credit_idio_only"],
            rtol=1e-6, atol=1e-9
        ), "Sum of credit_idio_only car_total does not equal sigma_credit_idio_only."

    return df_out

# ======================================================================
# Examples / Demo
# ======================================================================

def _build_toy_inputs() -> tuple[pd.DataFrame, pd.DataFrame, pd.DataFrame, pd.Series, pd.Series]:
    factors = ["Credit_Spread", "Credit_Curve", "Liquidity", "Rates_10y", "FX_Dollar"]
    credit_mask = pd.Series([True, True, True, False, False], index=factors)

    Sigma = pd.DataFrame(
        [[0.040, 0.010, 0.000, 0.000, 0.000],
         [0.010, 0.030, 0.010, 0.000, 0.000],
         [0.000, 0.010, 0.020, 0.000, 0.000],
         [0.000, 0.000, 0.000, 0.050, 0.000],
         [0.000, 0.000, 0.000, 0.000, 0.060]],
        index=factors, columns=factors
    )

    nodes = ["Banks", "Utilities", "MBS"]

    beta_p_df = pd.DataFrame([
        [ 0.30, -0.10, 0.20,  0.05,  0.02],  # Banks
        [ 0.10,  0.00, 0.05, -0.02,  0.01],  # Utilities
        [ 0.15,  0.05, 0.10,  0.01, -0.03],  # MBS
    ], index=nodes, columns=factors)

    beta_b_df = pd.DataFrame([
        [ 0.25, -0.05, 0.10,  0.00,  0.00],  # Banks
        [ 0.12,  0.00, 0.05,  0.00,  0.00],  # Utilities
        [ 0.10,  0.02, 0.08,  0.00,  0.00],  # MBS
    ], index=nodes, columns=factors)

    # Node idiosyncratic SARs (orthogonal, node-additive)
    sar_idio = pd.Series({
        "Banks": 0.80,
        "Utilities": 0.40,
        "MBS": 0.50,
    })

    return beta_p_df, beta_b_df, Sigma, credit_mask, sar_idio

def _run_demo():
    beta_p_df, beta_b_df, Sigma, credit_mask, sar_idio = _build_toy_inputs()

    df = build_credit_idio_only_table(
        portfolio_exposure_by_node_df=beta_p_df,
        benchmark_exposure_by_node_df=beta_b_df,
        factor_covariance_df=Sigma,
        credit_factor_mask=credit_mask,
        idiosyncratic_sar_by_node=sar_idio,
        add_totals_row=True,
        audit_checks=True,
    )

    print("Column blocks:", list(df.columns.levels[0]))
    print("Portfolio summary:", df.attrs.get("portfolio_summary", {}))
    print(df)

    # Mechanical checks
    # 1) Node-level: car_total == car_credit + car_idio
    np.testing.assert_allclose(
        df[("credit_idio_only","car_total")].to_numpy(),
        df[("credit_idio_only","car_credit")].to_numpy() + df[("credit_idio_only","car_idio")].to_numpy(),
        rtol=1e-6, atol=1e-9
    )
    # 2) Node-level: car_credit == car_beta + car_res
    np.testing.assert_allclose(
        df[("credit_idio_only","car_credit")].to_numpy(),
        df[("credit_idio_only","car_beta")].to_numpy() + df[("credit_idio_only","car_res")].to_numpy(),
        rtol=1e-6, atol=1e-9
    )

    print("\nDemo checks passed.")

if __name__ == "__main__":
    _run_demo()
